In [1]:
## imports

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import time
import pprint
from datetime import datetime
from pandas import ExcelWriter
from pandas import ExcelFile

In [2]:
# obtain read in excel file

df_url = pd.read_excel("/Users/Niall-McNulty/Desktop/Computer Science Projects:Courses/Web Scraping/Web-scraping-www.promodescuentos.com/promodescuentos.xlsx", index_col=0)


In [3]:
# create a list of URLS to iterate over
arr_url = [ x for x in df_url['URL']]
arr_url[:10]

['https://www.promodescuentos.com/ofertas/among-us-se-lanzara-con-xbox-game-pass-consola-el-14-de-diciembre-661506',
 'https://www.promodescuentos.com/ofertas/amazon-one-plus-7t-8gb-ram-128gb-renewed-661424',
 'https://www.promodescuentos.com/ofertas/curso-yo-se-excel-un-curso-de-excel-para-principiantes-661670',
 'https://www.promodescuentos.com/ofertas/casco-stormtrooper-starwars-con-visor-para-mala-punteria-661662',
 'https://www.promodescuentos.com/ofertas/grand-theft-auto-san-andreas-the-definitive-edition-en-game-pass-noviembre-11-661764',
 'https://www.promodescuentos.com/ofertas/iphone-12-pro-max-128gb-661724',
 'https://www.promodescuentos.com/ofertas/sorteo-buen-fin-2021-sat-661612',
 'https://www.promodescuentos.com/ofertas/soriana-hiper-y-super-folleto-productos-gratis-con-puntos-vigencia-de-puntos-recompensas-soriana-domingo-31-de-octubre-661108',
 'https://www.promodescuentos.com/ofertas/fallout-76-prueba-gratuita-multiplataforma-pc-steam-xbox-one-ps4-del-19-24-octubre-66

In [4]:

# create lists to append relevant data to

urls = []
top_comment_user = []
top_comment = []
thumbs_up = []

count = 0

# loop through urls and get data
for url in arr_url:
    
    
    # time test
    start = time.process_time()
    
    # append url to url array
    urls.append(url)

    # set up selenium / driver / path

    DRIVER_PATH = '/Users/Niall-McNulty/Desktop/Computer Science Projects:Courses/Web Scraping/chromedriver'
    # add headless mode
    options = Options()
    options.add_argument("--headless") # Runs Chrome in headless mode.
    options.add_argument('--no-sandbox') # Bypass OS security model
    driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
    driver.get(url)
   
    r = driver.page_source
    soup = BeautifulSoup(r, 'html.parser')
    
    # check if there is a top comments section
    if soup.find_all('span',{'class':'lbox--v-3 space--l-2 size--all-m size--fromW2-l text--b'}):
        find_comments = soup.find_all('span',{'class':'lbox--v-3 space--l-2 size--all-m size--fromW2-l text--b'})
        for elements in find_comments:
            # check for top comments
            if 'Mejores comentarios' in elements.text:
                # if there is, find the username (first matching element) and append to list
                if soup.find('span',{'class': 'userInfo-username'}).text:
                    user_name = soup.find('span',{'class': 'userInfo-username'}).text
                    top_comment_user.append(user_name)
                else:
                    top_comment_user.appned(None)

                # check for thumbs up amount and append to list
                if soup.find('span', {'class': 'comment-like'}).text:
                    thumbs_up_count = soup.find('span', {'class': 'comment-like'}).text
                    thumbs_up.append(thumbs_up_count)
                else:
                    thumbs_up.append(None)


                # check for the parent div for comments
                if soup.find('div',{'class':'commentList-item'}):
                    # assign it to a variable
                    parent = soup.find('div',{'class':'commentList-item'})
                    # check for top comment(first entry)
                    if parent.find('div',{'class':'comment-body'}):
                        # grab text
                        comment_text = parent.find('div',{'class':'comment-body'}).text
                        # if there is no text, it is assumed to be a graphic or image
                        if comment_text == '':
                            top_comment.append('Graphic instead of text (image/meme)')
                            # append text if there is
                        else:
                            top_comment.append(comment_text)
                else:
                    top_comment.append(None)


            else:
                top_comment.append(None)
        
    else:
        top_comment_user.append(None)
        top_comment.append(None)
        thumbs_up.append(None)
    
    # save every 500 entries incase of failure
    if(count%500==0):
        comment_dict = {'URL':urls,'top_comment_user':top_comment_user,'top_comment':top_comment,'thumbs_up':thumbs_up}
        comment_data = pd.DataFrame.from_dict(comment_dict)
        comment_data.to_csv("/Users/Niall-McNulty/Desktop/Computer Science Projects:Courses/Web Scraping/Web-scraping-www.promodescuentos.com/comment_data.csv", index = False)
    else:
        continue
    
    count += 1

    # close driver
    driver.quit()

comment_dict = {'URL':urls,'top_comment_user':top_comment_user,'top_comment':top_comment,'thumbs_up':thumbs_up}
comment_data = pd.DataFrame.from_dict(comment_dict)
comment_data.to_csv("/Users/Niall-McNulty/Desktop/Computer Science Projects:Courses/Web Scraping/Web-scraping-www.promodescuentos.com/comment_data.csv", index = False)

# code run time
print(time.process_time() - start)


KeyboardInterrupt: 

In [ ]:
comment_data